Dataset 6 : Working Memory 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.pipeline import Pipeline
from mne import Epochs, pick_types
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.io import concatenate_raws, read_raw_edf

print(__doc__)

tmin, tmax = -1.0, 4.0
subject = 1
runs = [6, 10, 14]  # motor imagery: hands vs feet

raw_fnames = eegbci.load_data(subject, runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
eegbci.standardize(raw) 
montage = make_standard_montage("standard_1005")
raw.set_montage(montage)
annotations = raw.annotations
print("Annotations", annotations)


Automatically created module for IPython interactive environment
Extracting EDF parameters from C:\Users\rebec\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\rebec\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\rebec\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Annotations <Annotations | 94 segments: BAD boundary (2), EDGE boundary (2), T0 (45), ...>


In [40]:
raw.annotations.rename(dict(T1="hands", T2="feet"))
raw.set_eeg_reference(projection=True)

# Apply band-pass filter
raw.filter(7.0, 30.0, fir_design="firwin", skip_by_annotation="edge")

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(
    raw,
    event_id=["hands", "feet"],
    tmin=tmin,
    tmax=tmax,
    proj=True,
    picks=picks,
    baseline=None,
    preload=True,
)
epochs_train = epochs.copy().crop(tmin=1.0, tmax=2.0)
labels = epochs.events[:, -1] - 2

<Annotations | 94 segments: BAD boundary (2), EDGE boundary (2), T0 (45), ...>

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


Measurement date,"August 12, 2009 16:15:00 GMT"
Experimenter,Unknown
Participant,X
Digitized points,67 points
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,160.00 Hz
Highpass,0.00 Hz
Lowpass,80.00 Hz


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Measurement date,"August 12, 2009 16:15:00 GMT"
Experimenter,Unknown
Participant,X
Digitized points,67 points
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,160.00 Hz
Highpass,7.00 Hz
Lowpass,30.00 Hz


Used Annotations descriptions: ['T0', 'feet', 'hands']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
45 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 45 events and 801 original time points ...
0 bad epochs dropped


In [44]:
# Import necessary libraries
import mne
import numpy as np
import pandas as pd
from mne.datasets import sleep_physionet
from mne.io import read_raw_edf
from mne import Epochs
# Compute PSD using Welch method
record = 'SC4001E0'  #
data_path = sleep_physionet.age.fetch_data(subjects=[0])  # Download the dataset
edf_path = data_path[0][0]  # Path to EEG data
hypnogram_path = data_path[0][1]  # Path to sleep stage annotations

# Set EEG reference
raw.set_eeg_reference(projection=True)

# Band-pass filter to keep relevant EEG frequencies
raw.filter(0.5, 30, fir_design='firwin')

# Retrieve annotations (sleep stages)
annotations = mne.read_annotations(hypnogram_path)

# Apply annotations to raw data
raw.set_annotations(annotations)

# Calculate Power Spectral Density (PSD)
psd, freqs = psd_welch(raw, fmin=0.5, fmax=30, tmin=0, tmax=None)

# Organize data for classification (e.g., average power in specific bands)
# Example: extracting delta, theta, alpha, and beta band power
delta = psd[(freqs >= 0.5) & (freqs <= 4)].mean(axis=0)
theta = psd[(freqs >= 4) & (freqs <= 8)].mean(axis=0)
alpha = psd[(freqs >= 8) & (freqs <= 13)].mean(axis=0)
beta = psd[(freqs >= 13) & (freqs <= 30)].mean(axis=0)

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVM

# Prepare feature matrix and labels
# Example: using PSD in different bands as features
X = np.stack([delta, theta, alpha, beta], axis=1)  # Feature matrix
y = annotations.description  # Sleep stage labels

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier (e.g., Random Forest)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Evaluate classifier with cross-validation
cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
print(f"Cross-validation accuracy: {np.mean(cv_scores)}")

# Test on the held-out test set
test_accuracy = clf.score(X_test, y_test)
print(f"Test set accuracy: {test_accuracy}")

Using default location ~/mne_data for PHYSIONET_SLEEP...


100%|#####################################| 48.3M/48.3M [00:00<00:00, 48.3GB/s]
100%|#############################################| 4.62k/4.62k [00:00<?, ?B/s]
100%|#####################################| 51.6M/51.6M [00:00<00:00, 49.3GB/s]
100%|#####################################| 4.58k/4.58k [00:00<00:00, 4.58MB/s]

Download complete in 04m17s (95.3 MB)
EEG channel type selected for re-referencing



C:\Users\rebec\AppData\Local\Temp\ipykernel_7640\3562866650.py:15: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference(projection=True)


Measurement date,"August 12, 2009 16:15:00 GMT"
Experimenter,Unknown
Participant,X
Digitized points,67 points
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,160.00 Hz
Highpass,7.00 Hz
Lowpass,30.00 Hz


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Measurement date,"August 12, 2009 16:15:00 GMT"
Experimenter,Unknown
Participant,X
Digitized points,67 points
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,160.00 Hz
Highpass,7.00 Hz
Lowpass,30.00 Hz


C:\Users\rebec\AppData\Local\Temp\ipykernel_7640\3562866650.py:24: RuntimeWarning: Omitted 153 annotation(s) that were outside data range.
  raw.set_annotations(annotations)
C:\Users\rebec\AppData\Local\Temp\ipykernel_7640\3562866650.py:24: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


Measurement date,"August 12, 2009 16:15:00 GMT"
Experimenter,Unknown
Participant,X
Digitized points,67 points
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,160.00 Hz
Highpass,7.00 Hz
Lowpass,30.00 Hz


NameError: name 'psd_welch' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, clf.predict(X_test))

# Plot confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=y_test.unique(), yticklabels=y_test.unique())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()